This Notebook solves the "Mackerel" problem in the Puzzler (https://fivethirtyeight.com/features/somethings-fishy-in-the-state-of-the-riddler/) of May 22, 2020.  A "mackerel" in this case is a word which has no letters in common with exactly one state.  "Mackerel" has no letters in common with "Ohio", but it does have letters in common with every other state.  The idea is to find: (a) the longest mackerel ; and (b) the state with the most mackerels.
Peter Norvig has a convenient list of all 263,534 words in the English Language.  So Step 1 is to get all of those words and sort them in descending order by length



In [25]:
import urllib.request
with urllib.request.urlopen('https://norvig.com/ngrams/word.list') as response:
   word_list_raw = response.read()
word_list_str = word_list_raw.decode()
word_list = word_list_str.split('\n')
word_list_sorted = sorted(word_list, reverse = True, key = lambda s: len(s))

Now that we have done that, define the states:

In [26]:
states=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

This is the principal class we'll use: it has a word (which should be all lower case), a set of the letters of that word, and, optionally, a state name.  Its principal function, given another WordSet, is to determine if there is any overlap between its letters and the letters of the other WordSet.  When creating the set, canonize it by lower-casing the word and removing any blanks (these occur in state names, e.g., North Dakota)

In [27]:
class WordSet:
  def __init__(self, word):
    self.word = word
    self.letterSet = set(word.lower()) - {' '}

  def overlap(self, otherSet):
    return bool(self.letterSet.intersection(otherSet.letterSet))

Convert the word_list and states into lists of WordSets

In [28]:
word_sets = [WordSet(word) for word in word_list_sorted]
state_sets = [WordSet(state) for state in states]

Does a word match only one state?  If it does, return it, otherwise return None

In [31]:
def match_only_one_state(word_set):
    matches = [state_set.word for state_set in state_sets if not word_set.overlap(state_set)]
    return matches[0] if len(matches) == 1 else None

Find all of the longest Mackerels and, for each one, the state it matches.

In [36]:
i = 0
for word_set in word_sets:
    i += 1
    mackerel_state = match_only_one_state(word_set)
    # If we have one, this is the longest mackerel!  Remember its length to get the ties
    if (mackerel_state):
        mackerels = [(word_set.word, mackerel_state)]
        mackerel_length = len(word_set.word)
        remaining_words = word_sets[i:]
        break
"Mackerel Length: %d" % mackerel_length

'Mackerel Length: 23'

Found the first and found the length, just find the ties.  Note we only have to look at words of length mackerel_length

In [37]:
for word_set in remaining_words:
    if (len(word_set.word) < mackerel_length):
        break
    mackerel_state = match_only_one_state(word_set)
    if (mackerel_state):
        mackerels.append((word_set.word, mackerel_state))
# Print them nicely
for mackerel in mackerels:
    print(', '.join(mackerel))

counterproductivenesses, Alabama
hydrochlorofluorocarbon, Mississippi


OK, find the state with the most mackerels.  Keep a count of mackerels by state, and update it as we go

In [44]:
mackerel_count = {}
for state in states: mackerel_count[state] = 0

OK, count them up.  Just iterate over the word_sets, and for each word_set if it's a mackerel for a state, add it in

In [45]:
for word_set in word_sets:
    mackerel_state = match_only_one_state(word_set)
    if (mackerel_state):
        mackerel_count[mackerel_state] += 1


'Result: Top Mackerel Count 11342, states: Ohio'

In [46]:
mackerel_list = [mackerel_count[state] for state in states]
max_mackerels = max(mackerel_list)
total_mackerels = sum(mackerel_list)
max_mackerel_states = [state for state in states if mackerel_count[state] == max_mackerels]
"Result: Total Mackerels: %d, Top Mackerel Count %d, states: %s" % (total_mackerels, max_mackerels, ', '.join(max_mackerel_states))

'Result: Total Mackerels: 45385, Top Mackerel Count 11342, states: Ohio'